In [1]:
import os
import shutil
import re
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import nibabel as nib
import matplotlib as mpl
import matplotlib.pyplot as plt

load_dotenv()

True

In [2]:
raw_dir = os.getenv("RAW")
nii_dir = os.getenv("NII")
print(raw_dir)

def moveRaw(raw_dir, nii_dir):
    """
    Function for converting files in the raw OAS directory to .nii, then moving them to the specified directory 'nii_dir'
    """
    i=0
    # Regex pattern to get the MRI id and visit number
    rawDirPat = r"(?:[\W\S]+?)OAS2_([0-9]{4})_MR([0-2]{1})/RAW"
    for root, dir, files in os.walk(raw_dir):
        r_match = re.findall(rawDirPat, root)
        if len(r_match) > 0:
            subID = r_match[0][0]
            session = r_match[0][1]
            new_name = f"{subID}_{session}"
            for f in files:
                # Get file extension
                fname, fext = os.path.splitext(f)
                # If the file is a .img, it should be converted to .nii
                if fext == ".img":
                    # Get mpr number from file name
                    f_match = re.findall(r"mpr-([0-2]{1}).nifti", fname)
                    if len(f_match) > 0:
                        f_num = f_match[0]

                        # Get full path of .img file
                        img_name = os.path.join(root, (fname + ".img"))

                        # Create new name for .nii file and put it in the right path
                        nii_name = os.path.join(
                            nii_dir, f"OAS2_{subID}_MR{session}_V{f_num}.nifti.nii"
                        )

                        # Load .img image using nibabel
                        img = nib.load(img_name)
                        # print(nii_name)

                        # Save .nii image in nii directory with new name
                        nib.save(img, nii_name)
                        print(f'{i:>4d}/1108 ({i*100./1108.:0>3.2f}%)')
                        # os.remove(os.path.join(root, fname + ".hdr"))
                        # os.remove(os.path.join(root, fname + ".img"))
                        i+=1




# moveRaw(raw_dir, nii_dir)

/home/Linux-Windows-Share/documents/biomed/classes/bmen-619/bmen-619-oasis-2/datasets/OAS2_RAW


Move all files to one directory

In [3]:
from sklearn.model_selection import train_test_split

def makeTVTSplit(df):
  x_train, x_temp, y_train, y_temp = train_test_split(df.drop(columns=["Group"]), df["Group"], test_size=0.2, stratify=df[['Group',"Sex_F"]])

  strat = pd.DataFrame(x_temp)
  strat["Group"] = y_temp

  x_test, x_val, y_test, y_val = train_test_split(x_temp, y_temp, stratify=strat[['Group',"Sex_F"]],test_size=0.2)

  # df_new = moveBackToSource(df, deleteInstead=True)
  train = x_train.copy(deep=True)
  train["Split"] = ["train"]*train.shape[0]
  train["Group"] = y_train.values

  validate = x_val.copy(deep=True)
  validate["Split"] = ["validate"]*validate.shape[0]
  validate["Group"] = y_val.values

  test = x_test.copy(deep=True)
  test["Split"] = ["test"]*test.shape[0]
  test["Group"] = y_test.values
  print(f"Split:")
  print(f'\tTest: {len(test)}')
  print(f'\tTrain: {len(train)}')
  print(f'\tValidate: {len(validate)}')

  df_new = pd.merge(train, test, how="outer")
  df_new = pd.merge(df_new, validate, how="outer")

  # df_new = moveByClass(df_new, removeOriginal=True)

  return df_new

df = pd.read_excel("OAS2-normalized.xlsx")
df_split = makeTVTSplit(df)
# df_split.to_excel("OAS2-split.xlsx")


Split:
	Test: 47
	Train: 232
	Validate: 12


In [4]:
def getImgGroup(filename, df):

  filePattern = r"(OAS2_[0-9]{4}_MR[0-9]{1})"
  r_match = re.match(filePattern, filename)
  split = df[df["MRI ID"]==r_match.groups()[0]]["Group"].values
  if len(split) == 0:
     return -1
  else:
     return split[0]


In [5]:
def toRGB(data):
  """
  Function for converting a numpy array to the proper format to be saved as an RGB image.

  Parameters
  ---------
  data : numpy.ndarray
      Array of data to be converted to image
  """

  x, y = data.shape[:2]
  data = (data-data.min())/(data.max()-data.min())
  img_arr = np.empty(shape=(x,y,4))
  img_arr[:, :, :3] = data
  img_arr[:, :, 3] = 1.
  return img_arr

def getImgSplit(filename, split_df):
  filePattern = r"(OAS2_[0-9]{4}_MR[0-9]{1})"
  r_match = re.match(filePattern, filename)
  split = split_df[split_df["MRI ID"]==r_match.groups()[0]]["Split"].values
  if len(split) == 0:
     return -1
  else:
     return split[0]

def convertToJPG(nii_dir, jpg_dir, sliceStart,split_df, numImages=1):
    i = 0
    shape = (0,0,0,0)

    for root, dir, files in os.walk(nii_dir):
        for f in files:
            fbase, fext = os.path.splitext(f)
            if fext == ".nii":
                # print(f"Converting {f}")
                i += 1
                fname = os.path.join(root, f)
                img = nib.load(fname)
                for i in range(numImages):
                  data = img.get_fdata()[sliceStart+i,:,:]
                  img_arr = toRGB(data)
                  shape = img_arr.shape
                  print(fbase)
                  split=getImgSplit(fbase, split_df)
                  group=getImgGroup(fbase, split_df)
                  if split!=-1 and group!=-1:
                    if split == 'test':
                      jpg_name = os.path.join(jpg_dir, split, f'{fbase.replace(".nifti","")}_{sliceStart+i}.jpg')
                    else:
                       jpg_name = os.path.join(jpg_dir, split, f'class_{group}', f'{fbase.replace(".nifti","")}_{sliceStart+i}.jpg')
                    plt.imsave(jpg_name, img_arr)
                # os.remove(os.path.join(root, fbase + ".nii"))

    return shape

jpg_dir = os.getenv("JPG")
if not os.path.exists(os.path.join(jpg_dir, "test")):
   os.makedirs(os.path.join(jpg_dir, "test"))

for split in ["validate", "train"]:
   if not os.path.exists(os.path.join(jpg_dir, split, "class_0")):
      os.makedirs(os.path.join(jpg_dir, split, "class_0"))
   if not os.path.exists(os.path.join(jpg_dir, split, "class_1")):
      os.makedirs(os.path.join(jpg_dir, split, "class_1"))


data_shape = (256, 128, 4)
# convertToJPG(nii_dir, jpg_dir, 135, df_split, 1)

# Model

In [6]:
import keras
from keras import layers
from keras import ops
from keras import Sequential
import tensorflow as tf
import SimpleITK as sitk


2025-04-09 01:30:17.199911: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744183817.213017  170329 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744183817.216975  170329 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744183817.227052  170329 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744183817.227070  170329 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744183817.227072  170329 computation_placer.cc:177] computation placer alr

In [7]:
resnet_base = keras.applications.ResNet50(
  include_top=False,
  weights='imagenet',
  input_shape=(data_shape[0], data_shape[1], 3)
)

resnet_base.trainable = False

model = Sequential([
  resnet_base,
  layers.Flatten(),
  layers.Dense(384, activation='relu'),
  layers.Dense(64, activation='relu'),
  layers.Dense(2, activation='softmax')
])

model.compile(
  optimizer=keras.optimizers.Adam(),
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy']
)


I0000 00:00:1744183818.857220  170329 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2270 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [8]:
def makeKerasTVTDatasets(batchSize):

  dset_train = keras.preprocessing.image_dataset_from_directory(
    directory=os.path.join(os.getenv("JPG"),"train"),
    seed=73,
    image_size=data_shape[:2],
    batch_size=batchSize,
    label_mode='binary')

  dset_validate = keras.preprocessing.image_dataset_from_directory(
    directory=os.path.join(os.getenv("JPG"),"validate"),
    seed=73,
    image_size=data_shape[:2],
    batch_size=batchSize,
    label_mode='binary')

  return dset_train, dset_validate

dset_train, dset_validate = makeKerasTVTDatasets(4)

Found 464 files belonging to 2 classes.
Found 24 files belonging to 2 classes.


In [9]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [10]:
# history = model.fit(dset_train, validation_data=dset_validate, epochs=15)

# Save the model
# model.save("oas2-model.keras")

model = keras.saving.load_model("oas2-model-final.keras")

In [11]:
predictions = []
fileList = []
mriID = []
groups = []
i = 0

df_old = df.copy(deep=True)


for root, dir, files in os.walk(os.getenv("JPG"), "test"):
  for f in files:
    g = getImgGroup(f, df)
    img = keras.preprocessing.image.load_img(os.path.join(root, f), target_size=data_shape[:2])
    img_arr = keras.preprocessing.image.img_to_array(img)
    img_arr = tf.expand_dims(img_arr, 0)
    groups += [g]
    mriID += [f[:13]]
    fileList += [f]
    predictions += [np.argmax(model.predict(img_arr))]
    print(i,'\n')
    i += 1

df_predictions = pd.DataFrame({"MRI ID": mriID, "Group": groups, "Prediction": predictions, "File": fileList})
df = pd.merge(df_predictions, df.copy(deep=True), how="outer")


I0000 00:00:1744183823.614878  170408 service.cc:152] XLA service 0x77db70049f20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744183823.614898  170408 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2025-04-09 01:30:23.680823: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1744183824.130815  170408 cuda_dnn.cc:529] Loaded cuDNN version 90300


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
0 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


I0000 00:00:1744183825.963563  170408 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
3 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
4 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
5 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
6 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
7 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
8 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
9 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
10 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
11 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
12 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
13 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
14 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
15 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
16 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
17 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
18 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
19 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
20 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
21 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
22 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
23 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
24 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
25 

1/1 ━━━━━━━━━━

In [18]:
df_predictions.to_excel("OAS2-predictions.xlsx")

In [12]:
print(df.shape)

tf_a = []
for index, row in df.iterrows():
  p = row["Prediction"]
  g = row["Group"]
  tf_a += [1 if g == p else 0]

df["TF"] = tf_a


# print(f"Accuracy: {100.*len(pred_true)/(len(pred_true) + len(pred_false))}")


(582, 15)


In [13]:
import math

def getRec(df):
  tp = float(len(df.query('TF == 1 and Prediction == 1')))
  fn = float(len(df.query('TF == 0 and Prediction == 0')))
  return tp/(fn+tp)

def getTPR(df):
  tp = float(len(df.query('TF == 1 and Prediction == 1')))
  fp = float(len(df.query('TF == 0 and Prediction == 1')))
  return tp/(tp+fp)

def getFPR(df):
  tp = float(len(df.query('TF == 1 and Prediction == 1')))
  fp = float(len(df.query('TF == 0 and Prediction == 1')))
  return fp/(tp+fp)

def getSpec(df):
  tn = len(df.query('TF == 1 and Prediction == 0'))
  fp = len(df.query('TF == 0 and Prediction == 1'))
  return float(tn)/(fp+tn)

def getAcc(df):
  return float(len(df[df["TF"]==1]))/len(df)

def getMCC(df):
  tp = float(len(df.query('TF == 1 and Prediction == 1')))
  tn = float(len(df.query('TF == 1 and Prediction == 0')))
  fp = float(len(df.query('TF == 0 and Prediction == 1')))
  fn = float(len(df.query('TF == 0 and Prediction == 0')))

  numer = (tp*tn)-(fp*fn)
  denom = math.sqrt((tp + fp)*(tp + fn)*(tn+fp)*(tn+fn))
  return (numer)/(denom)


def getEOD(df_a,df_b):
  rec_a = getRec(df_a)
  rec_b = getRec(df_b)
  return rec_a-rec_b

def getAOD(df_a,df_b):

  tpr_a = getTPR(df_a)
  tpr_b = getTPR(df_b)
  fpr_a = getFPR(df_a)
  fpr_b = getFPR(df_b)

  return ((fpr_b-fpr_a)+(tpr_b-tpr_a))/2.

def getDI(df_a, df_b):
  a = float(len(df_a[df_a["Prediction"]==1]))/len(df_a)
  b = float(len(df_b[df_b["Prediction"]==1]))/len(df_b)
  return a/b

def getSPD(df_a, df_b):
  a = float(len(df_a[df_a["Prediction"]==1]))/len(df_a)
  b = float(len(df_b[df_b["Prediction"]==1]))/len(df_b)
  return a-b

def getAccDiff(df_a, df_b, asPercent=False):
  a = getAcc(df_a)
  b = getAcc(df_b)
  if asPercent:
    return 100*(a-b)/b
  else:
    return a-b

def getRecDiff(df_a, df_b, asPercent=False):
  a = getRec(df_a)
  b = getRec(df_b)
  if asPercent:
    return 100*(a-b)/b
  else:
    return a-b

def getSpecDiff(df_a, df_b, asPercent=False):
  a = getSpec(df_a)
  b = getSpec(df_b)
  if asPercent:
    return 100*(a-b)/b
  else:
    return a-b

def getMCCDiff(df_a, df_b, asPercent=False):
  a=getMCC(df_a)
  b=getMCC(df_b)
  if asPercent:
    return 100*(a-b)/b
  else:
    return a-b

def printComparison(df_a, df_b, title_a="A", title_b="B"):
  print("-"*80)
  print(f"COMPARISON {title_a} vs. {title_b}")
  print("·"*40)
  print(f'Accuracy difference ({title_a} vs. {title_b}):   \t{getAccDiff(df_a, df_b , asPercent=True):+.2f}%')
  print(f'Recall difference ({title_a} vs. {title_b}):     \t{getRecDiff (df_a, df_b, asPercent=True):+.2f}%')
  print(f'Specificity difference ({title_a} vs. {title_b}):\t{getSpecDiff(df_a, df_b, asPercent=True):+.2f}%')
  print(f'MCC difference ({title_a} vs. {title_b}):        \t{getMCCDiff(df_a, df_b , asPercent=True):+.2f}%')
  print()
  print(f'Equal opportunity difference (EOD):\t{getEOD(df_a, df_b):+.2f}\t{"In ideal range" if (-0.1<=getEOD(df_a, df_b)<=0.1) else "Not in ideal range"}')
  print(f'Average odds difference (AOD):     \t{getAOD(df_a, df_b):+.2f}\t{"In ideal range" if (-0.1<=getAOD(df_a, df_b)<=0.1) else "Not in ideal range"}')
  print(f'Disparate impact (DI):             \t{getDI(df_a, df_b):+.2f}\t{"In ideal range" if (-0.1<=getDI(df_a, df_b)<=0.1) else "Not in ideal range"}')
  print(f'Statistical parity index (SPD):    \t{getSPD(df_a, df_b):+.2f}\t{"In ideal range" if (-0.1<=getSPD(df_a, df_b)<=0.1) else "Not in ideal range"}')
  print("-"*80)


In [14]:
print(f"Recall:     \t{getRec(df):.3f}")
print(f"Accuracy:   \t{getAcc(df):.3f}")
print(f"Specificity:\t{getSpec(df):.3f}")
print(f"MCC:        \t{getMCC(df):.3f}")

print()

printComparison(df[df["Sex_F"]==1], df[df["Sex_F"]==0], "F", "M")



# printComparison(df[df["Age"]<=0.5], df[df["Age"]>0.5], "Age in bottom 50%", "Age in top 50%")


Recall:     	0.907
Accuracy:   	0.924
Specificity:	0.938
MCC:        	0.845

--------------------------------------------------------------------------------
COMPARISON F vs. M
········································
Accuracy difference (F vs. M):   	-0.23%
Recall difference (F vs. M):     	-8.95%
Specificity difference (F vs. M):	+5.66%
MCC difference (F vs. M):        	-3.18%

Equal opportunity difference (EOD):	-0.08	In ideal range
Average odds difference (AOD):     	-0.00	In ideal range
Disparate impact (DI):             	+0.51	Not in ideal range
Statistical parity index (SPD):    	-0.29	Not in ideal range
--------------------------------------------------------------------------------


In [15]:
img = keras.preprocessing.image.load_img("/home/Linux-Windows-Share/documents/biomed/classes/bmen-619/bmen-619-oasis-2/datasets/OAS2_JPG/train/class_0/OAS2_0145_MR2_V2_135.jpg", target_size=data_shape[:2])
img_arr = keras.preprocessing.image.img_to_array(img)
img_arr = tf.expand_dims(img_arr, 0)
print(np.argmax(model.predict(img_arr)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
0


In [16]:
img = keras.preprocessing.image.load_img("/home/Linux-Windows-Share/documents/biomed/classes/bmen-619/bmen-619-oasis-2/datasets/OAS2_JPG/train/class_1/OAS2_0075_MR1_V2_135.jpg", target_size=data_shape[:2])
img_arr = keras.preprocessing.image.img_to_array(img)
img_arr = tf.expand_dims(img_arr, 0)
print(np.argmax(model.predict(img_arr)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1
